# **Punto 3**

Modelos de Regresión. Considere el conjunto de datos Wind Speed. Implemente la versión de regresión de cada uno de los modelos estudiados en clases, para predecir velocidad del viento horaria (VENTO,
VELOCIDADE HORARIA (m/s)) en el conjunto de datos suministrado.

Luego de hacer el análisis exploratorio de las variables, encontramos al final cual seria nuestro conjunto final para entrenar y testear nuestros modelos que en este caso serán `K-NN`, `Regresion lineal`, `Regresion Ridge` y `Regresion Lasso`.

In [76]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox


In [2]:
wind=pd.read_csv("C:/Users/fonta/OneDrive/Documentos/CDD/6. Sexto semestre/Machine learning/Segundo parcial/wind_speed.csv")
wind.head()

,hora,dir_viento,vel_viento,humedad_max,humedad_min,temperatura_max,temperatura_min,precipita_total,rafagamax_viento,presionatm_max,presionatm_min
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,0.0,5.8,887.4,886.5


In [3]:
wind.drop('hora',axis=1,inplace=True)

Para poder dar solución a lo propuesto en el enunciado de este parcial, se debe hacer de manera manual la validación cruzada y la hiperparametización de cada uno de los modelos, a continuacion se definirán las funciones que nos ayudaran a lograr desplegar los modelos.

**Conjuntos de entrenamiento y test**

In [5]:
X_wind=wind.drop('vel_viento',axis=1)
Y=wind[['vel_viento']]

**Funcion que se encarga de separar los pliegues en la validación cruzada**

In [6]:
def entrenamiento_test(X, Y, start, T, h):
    x_train = X[start : start + T * 24]
    y_train = Y[start : start + T * 24]
    
    x_test = X[start + T * 24 + h : start + T * 24 + h + 24]
    y_test = Y[start + T * 24 + h : start + T * 24 + h + 24]
    
    return x_train, y_train, x_test, y_test

**Definicion de el dataframe con los resultados de los 24 modelos**

In [77]:
def dataframe_resultados(h, mejor_ventana, modelos):
    nuevo_modelo = pd.DataFrame({
        'Hora': [h],
        'Ventana': [mejor_ventana[0]],
        'Mejor_parametro': [mejor_ventana[1]],
        'R2': [mejor_ventana[2]],
        'MSE': [mejor_ventana[3]],
        'MAE': [mejor_ventana[4]],
        'RMSE': [mejor_ventana[5]]
    })
    
    modelos = pd.concat([modelos, nuevo_modelo], ignore_index=True)
    return modelos


Con esto definido podemos pasar a implementar cada uno de los modelos.

## **K-NN**

Como sabemos, debemos implentar 24 modelos, donde cada uno de ellos, busca predecir cada uno de los siguientes días, veamos como se comporta el modelo k-nn para cada una de las predicciones.

**Funcion que se encarga de hiperparametrizar el modelo en cada uno de los pliegues de la validación.**

In [80]:
def obtener_mejor_parametro_k(x_train, y_train, x_test, y_test, parametros):
    mejor_r2 = -np.inf
    mejor_k = None
    mejor_mse = None
    mejor_mae = None
    mejor_mape = None
    
    for k in parametros:
        knn = KNeighborsRegressor(n_neighbors=k)
        knn.fit(x_train, y_train)
        
        y_pred = knn.predict(x_test)
        score_r2 = r2_score(y_test, y_pred)
        
        # Calcular las métricas adicionales
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        # Actualizar el mejor k si este R² es mayor
        if score_r2 > mejor_r2:
            mejor_r2 = score_r2
            mejor_k = k
            mejor_mse = mse
            mejor_mae = mae
            mejor_rmse = rmse
    
    return mejor_k, mejor_r2, mejor_mse, mejor_mae, mejor_rmse

**Ciclo principal, se despliegan los 24 modelos, cada uno con su validación cruzada e hiperparametrización.**

In [ ]:
scaler = MinMaxScaler()
modelos_knn = pd.DataFrame(columns=['Hora', 'Ventana', 'Mejor_parametro', 'R2', 'MSE','MAE','RMSE'])
ventanas = [7, 14, 21, 28]
vecinos = [5, 10, 15, 20, 25, 30, 50]

for h in range(1, 25):
    mejores_k_por_ventana = []

    for T in ventanas:
        start = 0
        while start + (T * 24 + h) < len(X_wind):
            x_train, y_train, x_test, y_test = entrenamiento_test(X_wind, Y, start, T, h)
            
            x_train_norm = scaler.fit_transform(x_train)
            x_test_norm = scaler.transform(x_test)

            mejor_k, mejor_r2, mejor_mse, mejor_mae, mejor_mape = obtener_mejor_parametro_k(x_train_norm, y_train, x_test_norm, y_test, vecinos)
            
            mejores_k_por_ventana.append((T, mejor_k, mejor_r2, mejor_mse, mejor_mae, mejor_mape))

            start += T * 24
    
    
    mejor_ventana = max(mejores_k_por_ventana, key=lambda x: x[2])
    
    modelos_knn = dataframe_resultados(h, mejor_ventana, modelos_knn)
modelos_knn=modelos_knn.rename(columns={'Mejor_parametro': 'Mejor k'})
modelos_knn.to_csv('modelos_knn.csv',index=False)


### **Resultados**

In [66]:
modelos_knn

,Hora,Ventana,Mejor k,R2,MSE,MAE,RMSE
0,1,14,10,0.888929,0.158854,0.330417,0.398565
1,2,14,10,0.885492,0.156492,0.326667,0.395590
2,3,21,5,0.880477,0.140033,0.291667,0.374210
3,4,14,10,0.879754,0.175908,0.348333,0.419414
4,5,28,15,0.883015,0.124733,0.288333,0.353176
5,6,7,5,0.892518,0.224450,0.384167,0.473762
6,7,21,10,0.892036,0.133058,0.254167,0.364772
7,8,7,5,0.885964,0.212383,0.372500,0.460851
8,9,21,30,0.882219,0.135626,0.253750,0.368275
9,10,14,10,0.883813,0.196096,0.322917,0.442827


En la tabla enterior vemos un dataframe, donde cada fila indica los scores obtenidos para cada uno de lo 24 modelos, así como cual fue la ventana de predicción que arrojo este score que es el mejor dentro de las demas ventanas y por supuesto su parametro k respectivo, ademas, podemos evaluar como se comporta cada modelo basado en sus errores.

Por ejemplo si suponemos que nos intereza evaluar el modelo que predice el día número 10 vemos que la mejor ventana es la de 14 días con un parametro $k=10$, esta combinación de parametros tiene una explicación de la variabilidad de los datos ($R^2$) del 89% y además vemos que los errores (MAE, MSE, MAPE) son bastante pequeños que es lo ideal.

In [67]:
fig = go.Figure()

x=['7','14','28','21']
ventanas=modelos_knn['Ventana'].value_counts()
fig.add_trace(go.Bar(x=x,y=ventanas, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores modelos K-NN',
    xaxis_title='Ventanas de predicción',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Podemos ayudarnos de la gráfica anterior para darnos una idea de cual es la ventana que mas se usó para predecir cada uno de los días, en este caso es la ventana de 7 días con una frecuencia de 9, revisemos ahora de una manera similar, cuales fueron los mejores parametros que se escogieron.

In [70]:
fig = go.Figure()

x=['15','10','5','20',30]
ventanas=modelos_knn['Mejor k'].value_counts()
fig.add_trace(go.Bar(x=x,y=ventanas, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores modelos K-NN',
    xaxis_title='Ventanas de predicción',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Con la gráfica anterior podemos identificar la frecuencia con la que fueron elegidos los parametros para cada uno de los 24 modelos, vemos que el mas frecuente es el parametro $k=15$. 

Podemos ahora análisar los promedios de cada una de las métricas de error en cada una de las ventanas de predicción, vara así hacer una conclusión general.

#### ***Promedio por ventana***

In [71]:
columnas_metricas = ['R2', 'MSE', 'MAE','RMSE']
prom_knn = modelos_knn.groupby('Ventana')[columnas_metricas].mean().reset_index()
prom_knn

,Ventana,R2,MSE,MAE,RMSE
0,7,0.900974,0.164615,0.317091,0.402234
1,14,0.888233,0.171861,0.318859,0.413146
2,21,0.884910,0.136239,0.266528,0.369086
3,28,0.890398,0.207283,0.327389,0.452559


Como dijimos anteriormente, mediante la tabla podemos ver le promedio de las métricas para cada una de las ventanas, vemos que el $r^2$ promedio mas grande esta en la ventana de 7 días con un ajuste de 90%, las demas ventanas tienen un ajuste promedio de 88%, lo cual indica que el modelo k-nn se ajusta bien a la variabilidad de los datos, ademas los errores son bajos en cada una de las ventanas, lo cual es lo esperado.

## **Ridge**

En esta sección se implementará la misma lógica del modelo anterior, veamos.

In [11]:
def obtener_mejor_alpha_rige(x_train, y_train, x_test, y_test, parametros):
    mejor_r2 = -np.inf
    mejor_alpha= None
    mejor_mse = None
    mejor_mae = None
    mejor_mape = None
    
    for alpha in parametros:
        ridge = Ridge(alpha=alpha)
        ridge.fit(x_train, y_train)
        
        y_pred = ridge.predict(x_test)
        score_r2 = r2_score(y_test, y_pred)
        
        # Calcular las métricas adicionales
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        # Actualizar el mejor k si este R² es mayor
        if score_r2 > mejor_r2:
            mejor_r2 = score_r2
            mejor_alpha = alpha
            mejor_mse = mse
            mejor_mae = mae
            mejor_rmse = rmse
    
    return mejor_alpha, mejor_r2, mejor_mse, mejor_mae, mejor_rmse

En este ciclio principal buscaremos los mejores parametros del modelo rige para cada uno de los modelos para cada día, obteniendo como resultado los mejores parámetros y las métricas en cada modelo.

In [14]:
scaler = MinMaxScaler()
modelos_ridge = pd.DataFrame(columns=['Hora', 'Ventana', 'Mejor_parametro', 'R2', 'MSE','MAE','RMSE'])
ventanas = [7, 14, 21, 28]
alpha = [0.001, 0.01, 0.1, 1.0, 10.0] 

for h in range(1, 25):
    mejores_alpha_por_ventana = []

    for T in ventanas:
        start = 0
        while start + (T * 24 + h) < len(X_wind):
            x_train, y_train, x_test, y_test = entrenamiento_test(X_wind, Y, start, T, h)
            
            x_train_norm = scaler.fit_transform(x_train)
            x_test_norm = scaler.transform(x_test)

            mejor_alpha, mejor_r2, mejor_mse, mejor_mae, mejor_rmse = obtener_mejor_alpha_rige(x_train_norm, y_train, x_test_norm, y_test, alpha)
            
            mejores_alpha_por_ventana.append((T, mejor_alpha, mejor_r2, mejor_mse, mejor_mae, mejor_rmse))

            start += T * 24
    
    
    mejor_ventana = max(mejores_alpha_por_ventana, key=lambda x: x[2])
    
    modelos_ridge = dataframe_resultados(h, mejor_ventana, modelos_ridge)

# Mostrar el DataFrame final con los mejores modelos
modelos_ridge=modelos_ridge.rename(columns={'Mejor_parametro': 'Mejor Alpha'})
modelos_ridge.to_csv('modelos_ridge.csv', index=False)


C:\Users\fonta\AppData\Local\Temp\ipykernel_41148\3951791870.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelos = pd.concat([modelos, nuevo_modelo], ignore_index=True)


### **Resultados**

In [16]:
modelos_ridge

,Hora,Ventana,Mejor Alpha,R2,MSE,MAE,RMSE
0,1,21,0.001,0.934592,0.329656,0.427673,0.574157
1,2,21,0.100,0.929795,0.330341,0.431771,0.574753
2,3,21,1.000,0.944250,0.065317,0.201524,0.255571
3,4,21,1.000,0.941317,0.065692,0.203539,0.256304
4,5,21,1.000,0.939059,0.066881,0.207506,0.258615
5,6,21,1.000,0.937825,0.064859,0.198789,0.254675
6,7,21,1.000,0.939630,0.074402,0.215061,0.272767
7,8,7,0.001,0.935651,0.150246,0.328315,0.387616
8,9,7,1.000,0.937373,0.083394,0.226741,0.288780
9,10,7,1.000,0.937972,0.085158,0.233906,0.291818


Vemos que obtenemos una tabla parecida a la que conseguimos en el modelo anterior, si damos una revision un poco rápida, podemos ver que en general estos modelo Ridge, tienen un mejor ajuste en comparación con el modelo anterior, ademas los errores son mas pequeños.

Veamos ahora el conteo de las ventanas de predicción a ver cual es la mas usada para predecir.

In [26]:
fig = go.Figure()

x=['7','21','14']
ventanas=modelos_ridge['Ventana'].value_counts()
fig.add_trace(go.Bar(x=x,y=ventanas, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores modelos Ridge',
    xaxis_title='Ventanas de predicción',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Vemos que la ventana que mas fue escogida por nuestro programa es la ventana de 7 días, ademas vemos que no se tiene presencia de la ventana de 28 días.

Veamos que parametros fueron los mas usados.

In [25]:
fig = go.Figure()

x=['1.000','0.001','0.100','0.010' ]
alphass=modelos_ridge['Mejor Alpha'].value_counts()
fig.add_trace(go.Bar(x=x,y=alphass, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores parametros (Alpha) Ridge',
    xaxis_title='Pametros',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Con la figura anterior, vemos de anera gráfica cuales fueron los parametros mas escogidos para nuestras predicciones de interés, vemos que el mas usado es $\alpha=1$. 

#### ***Promedio por ventanas***

In [31]:
columnas_metricas = ['R2', 'MSE', 'MAE','RMSE']
prom_ridge = modelos_ridge.groupby('Ventana')[columnas_metricas].mean().reset_index()
prom_ridge

,Ventana,R2,MSE,MAE,RMSE
0,7,0.936708,0.106149,0.274262,0.324560
1,14,0.927416,0.132405,0.278138,0.363874
2,21,0.938067,0.142450,0.269409,0.349549


Esta ultima tabla nos ayuda a conluir de mejor forma como se esta comportando el modelo Ridge para nuestras predicciones de interés, vemos que el promedio del ajuste de la variabilidad de los datos es bastante bueno en cada una de las ventanas, vemos que el error absoluto medio es bastante pequeño, lo que quiere decir que las predicciones no estan tan alejadas de los datos reales, así que podemos decir que en general, el modelo ridge ajusta muy bien en cada una de las ventanas, siendo el mejor horizonte la ventana de 21 con un $r^2=93.8$ %

## **Lasso**

Pasemos a desplegar y evaluar el modelo Lasso, para nuestras predicciones de interés:

In [32]:
def obtener_mejor_alpha_lasso(x_train, y_train, x_test, y_test, parametros):
    mejor_r2 = -np.inf
    mejor_alpha= None
    mejor_mse = None
    mejor_mae = None
    mejor_mape = None
    
    for alpha in parametros:
        lasso = Lasso(alpha=alpha)
        lasso.fit(x_train, y_train)
        
        y_pred = lasso.predict(x_test)
        score_r2 = r2_score(y_test, y_pred)
        
        # Calcular las métricas adicionales
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        # Actualizar el mejor k si este R² es mayor
        if score_r2 > mejor_r2:
            mejor_r2 = score_r2
            mejor_alpha = alpha
            mejor_mse = mse
            mejor_mae = mae
            mejor_rmse = rmse
    
    return mejor_alpha, mejor_r2, mejor_mse, mejor_mae, mejor_rmse

In [38]:
scaler = MinMaxScaler()
modelos_lasso = pd.DataFrame(columns=['Hora', 'Ventana', 'Mejor_parametro', 'R2', 'MSE','MAE','RMSE'])
ventanas = [7, 14, 21, 28]
alpha = [0.001, 0.01, 0.1, 1.0, 10.0] 

for h in range(1, 25):
    mejores_alpha_por_ventana = []

    for T in ventanas:
        start = 0
        while start + (T * 24 + h) < len(X_wind):
            x_train, y_train, x_test, y_test = entrenamiento_test(X_wind, Y, start, T, h)
            
            x_train_norm = scaler.fit_transform(x_train)
            x_test_norm = scaler.transform(x_test)

            mejor_alpha, mejor_r2, mejor_mse, mejor_mae, mejor_rmse = obtener_mejor_alpha_lasso(x_train_norm, y_train, x_test_norm, y_test, alpha)
            
            mejores_alpha_por_ventana.append((T, mejor_alpha, mejor_r2, mejor_mse, mejor_mae, mejor_rmse))

            start += T * 24
    
    
    mejor_ventana = max(mejores_alpha_por_ventana, key=lambda x: x[2])
    
    modelos_lasso = dataframe_resultados(h, mejor_ventana, modelos_lasso)

# Mostrar el DataFrame final con los mejores modelos
modelos_lasso=modelos_lasso.rename(columns={'Mejor_parametro': 'Mejor Alpha'})
modelos_lasso.to_csv('modelos_lasso.csv', index=False)


C:\Users\fonta\AppData\Local\Temp\ipykernel_41148\3951791870.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



### **Resultados**

In [39]:
modelos_lasso

,Hora,Ventana,Mejor Alpha,R2,MSE,MAE,RMSE
0,1,21,0.001,0.934778,0.328715,0.430094,0.573337
1,2,21,0.001,0.930312,0.327912,0.429346,0.572636
2,3,21,0.010,0.937317,0.073439,0.222907,0.270996
3,4,21,0.010,0.934820,0.072964,0.220274,0.270118
4,5,21,0.010,0.932197,0.074412,0.223914,0.272786
5,6,7,0.001,0.933201,0.162489,0.339456,0.403100
6,7,14,0.001,0.933864,0.091116,0.227098,0.301854
7,8,14,0.001,0.932338,0.126903,0.267321,0.356235
8,9,14,0.001,0.931881,0.126356,0.265645,0.355467
9,10,14,0.001,0.932090,0.093233,0.238569,0.305341


En la tabla anterior se muestran los resultados del modelo lasso para cada una de los días de interés, vemos de manera muy general que los resultados de este modelo no son muy distintos a los del modelo anterior, la mayor diferencia entre Ridge y Lasso es que en cuestiones de tiempo ocupado para hacer las predicciones lasso tardó un total de 7 minutos, a diferencia del modelo Ridge y K-NN que tardaron 15 y 24 minutos respectivamente.

In [45]:
fig = go.Figure()

x=['14','7','21']
ventanas=modelos_lasso['Ventana'].value_counts()
fig.add_trace(go.Bar(x=x,y=ventanas, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores modelos Lasso',
    xaxis_title='Ventanas de predicción',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Con la gráfica anterior ilustramos que el modelo que fue mas escogido por nuestro programa es la ventana de 14 días y vemos que nuevamente la ventana de 28 días no fue elegida para ninguno de los 24 modelos.

In [43]:
fig = go.Figure()

x=['0.001','0.010' ]
alphass=modelos_lasso['Mejor Alpha'].value_counts()
fig.add_trace(go.Bar(x=x,y=alphass, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores parametros (Alpha) Ridge',
    xaxis_title='Pametros',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Para el caso de este modelo solo fueron escogido dos de los 5 parametros propuestos para la hiperparametrización, siendo el mas escogido el $\alpha=0.001$ con una frecuencia de 17.

Pasemos a analisar como lo hemos venido haciendo, los premedios de las ventanas que fueron tomadas para el modelo Lasso.

#### ***Promedio por ventana***

In [44]:
columnas_metricas = ['R2', 'MSE', 'MAE','RMSE']
prom_lasso = modelos_lasso.groupby('Ventana')[columnas_metricas].mean().reset_index()
prom_lasso

,Ventana,R2,MSE,MAE,RMSE
0,7,0.935282,0.132951,0.306613,0.363776
1,14,0.932136,0.116052,0.263656,0.340057
2,21,0.933885,0.175488,0.305307,0.391975


Con la tabla anterior podemos concluir que el promedio de cada una de las ventanas de prediccioón no paerecen tener diferencias significativas, al igual que los errores. Así que en general podemos decir que el modelo lasso ajusta muy bien tambien a nuestros datos, además el mejor $r^2$ promedio esta en la ventana de 7 días y es de 93.52% que si recordamos el mejor $R^2$ promedio de Ridge este fue de 93.8% así que Ridge sigue siendo hasta ahora el mejor modelo en terminos del promedio.

## **Regresión lineal**

In [50]:
def dataframe_resultados_rl(h, mejor_ventana, modelos):
    nuevo_modelo = pd.DataFrame({
        'Hora': [h],
        'Ventana': [mejor_ventana[0]],
        'R2': [mejor_ventana[1]],
        'MSE': [mejor_ventana[2]],
        'MAE': [mejor_ventana[3]],
        'RMSE': [mejor_ventana[4]]
    })
    
    modelos = pd.concat([modelos, nuevo_modelo], ignore_index=True)
    return modelos

In [52]:
def obtener_metricas_regresion_lineal(x_train, y_train, x_test, y_test):
    modelo = LinearRegression()
    modelo.fit(x_train, y_train)
    
    y_pred = modelo.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    return r2, mse, mae, rmse

In [55]:
scaler = MinMaxScaler()
modelos_lineal = pd.DataFrame(columns=['Hora', 'Ventana', 'R2', 'MSE', 'MAE', 'RMSE'])
ventanas = [7, 14, 21, 28]

for h in range(1, 25):
    mejores_metricas_por_ventana = []

    for T in ventanas:
        start = 0
        while start + (T * 24 + h) < len(X_wind):
            x_train, y_train, x_test, y_test = entrenamiento_test(X_wind, Y, start, T, h)
            
           
            x_train_norm = scaler.fit_transform(x_train)
            x_test_norm = scaler.transform(x_test)

            mejor_r2, mejor_mse, mejor_mae, mejor_rmse = obtener_metricas_regresion_lineal(x_train_norm, y_train, x_test_norm, y_test)
            
            mejores_metricas_por_ventana.append((T, mejor_r2, mejor_mse, mejor_mae, mejor_rmse))

            start += T * 24
    
    
    mejor_ventana = max(mejores_metricas_por_ventana, key=lambda x: x[1])
    
   
    modelos_lineal = dataframe_resultados_rl(h, mejor_ventana, modelos_lineal)

# Mostrar el DataFrame final con los mejores modelos
modelos_lineal.to_csv('modelos_lineal.csv', index=False)

C:\Users\fonta\AppData\Local\Temp\ipykernel_41148\638830773.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



### **Resultados**

In [56]:
modelos_lineal

,Hora,Ventana,R2,MSE,MAE,RMSE
0,1,21,0.934592,0.329655,0.427635,0.574156
1,2,21,0.929464,0.331898,0.429540,0.576106
2,3,21,0.929322,0.082806,0.237134,0.287760
3,4,21,0.927122,0.081582,0.231659,0.285625
4,5,21,0.924181,0.083210,0.235198,0.288461
5,6,7,0.936570,0.154294,0.330820,0.392802
6,7,7,0.937148,0.154607,0.333311,0.393201
7,8,7,0.935742,0.150032,0.328043,0.387340
8,9,14,0.930294,0.092820,0.219539,0.304664
9,10,14,0.931310,0.094304,0.226832,0.307089


Según vemos en la tabla de resultados, el modelo de regresion lineal ajusta igual de bien que los modelos de Redge y Lasso que a su vez son mejores que el modelo K-NN, verifiquemos esto con los siguientes análisis.

In [58]:
fig = go.Figure()

x=['7','14','21']
ventanas=modelos_ridge['Ventana'].value_counts()
fig.add_trace(go.Bar(x=x,y=ventanas, marker_color='#473C8B'))

fig.update_layout(
    title='Conteo de mejores modelos Ridge',
    xaxis_title='Ventanas de predicción',
    yaxis_title='',
    width=800,   
    height=500   
)

fig.show()

Vemos que para el modelo lineal, la ventana que mas fue escogida es la ventana de 7 días al igual que en los modelos Ridge y K-NN. Veamos como es el promedio de las métricas por cada ventana de predicción.

#### ***Promedio por ventana***

In [60]:
columnas_metricas = ['R2', 'MSE', 'MAE','RMSE']
prom_lineal = modelos_lineal.groupby('Ventana')[columnas_metricas].mean().reset_index()
prom_lineal

,Ventana,R2,MSE,MAE,RMSE
0,7,0.936751,0.123015,0.300125,0.349301
1,14,0.928698,0.112422,0.254797,0.334379
2,21,0.928936,0.181830,0.312233,0.402422


Podemos conluir que esta tabla del promedio de métricas para el modelo lineal es muy parecida a las obtenidas en los modelos de Ridge y Lasso. El ajuste del modelo es muy bueno y los errores son pequeños lo cual es ideal al momento de evaluar el funcionamiento del modelo.